# Bulk download fic pdfs

In [41]:
import os
import json
import re
import sys
sys.path.append("..")
import requests
import pandas as pd
from time import time
import logging
from multiprocessing.pool import ThreadPool

In [42]:
pwd

'/Users/yuchen.zhang/Documents/Projects/Class_Action/class-action-git/src'

Load and clean data

In [43]:
# Load input data and clean
scraped_data = pd.read_csv("../../data/data_from_scrapy.csv")
scraped_fic = scraped_data[scraped_data['fic_summary_table'].notna()]
scraped_fic["docket"][scraped_fic["docket"].isna()] = "na"
scraped_fic["court"][scraped_fic["court"].isna()] = "na"
len(scraped_fic)

/Users/yuchen.zhang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/yuchen.zhang/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:8767: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/Users/yuchen.zhang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

5254

Create Unique ID

In [90]:
# Unique Stanford Id and check if unique
scraped_fic['stfid'] = scraped_fic.apply(lambda row: row.url[row.url.find("id=")+len("id="):], axis = 1)
print(len(scraped_fic['stfid'].unique()) == len(scraped_fic))

# Create ids
scraped_fic["cd"] = scraped_fic["court"] + "_" + scraped_fic["docket"]
scraped_fic["id"] = scraped_fic["stfid"] + "_" + scraped_fic["cd"]

# Output intermed data with unique id
scraped_fic.to_csv("../../data/data_fic_id.csv")

/Users/yuchen.zhang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/yuchen.zhang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/yuchen.zhang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

True


Sample Data

In [45]:
scraped_fic.head(2)

,case_brief,case_name,case_status,class_period_end,class_period_start,company_market,court,date_filed,date_of_last_review,docket,...,industry,judge,market_status,plaintiffs,sector,ticker_symbol,url,stfid,cd,id
0,"According to the Complaint, Wirecard is a tech...",Wirecard AG Securities Litigation,ONGOING,6/24/20,8/17/15,OTC-BB,E.D. Pennsylvania,7/7/20,7/8/20,20-CV-03326,...,Software & Programming,Hon. ANITA B. BRODY,Public (Listed),"The Rosen Law Firm, P.A. (Jenkintown)",Technology,WCAGY,http://securities.stanford.edu/filings-case.ht...,107457,E.D. Pennsylvania_20-CV-03326,107457_E.D. Pennsylvania_20-CV-03326
1,"According to the Complaint, Finjan Holdings, I...","Finjan Holdings, Inc. Securities Litigation",ONGOING,6/29/20,6/10/20,NASDAQ,N.D. California,6/29/20,6/30/20,20-CV-04289,...,Software & Programming,Hon. Edward M. Chen,Public (Listed),"Brodsky & Smith, LLC (California)",Technology,FNJN,http://securities.stanford.edu/filings-case.ht...,107453,N.D. California_20-CV-04289,107453_N.D. California_20-CV-04289


In [169]:
pd.read_json(scraped_fic["fic_summary_table"][0])

,No,Document Title,Filing Date,link
0,1,Complaint - Class Action,07/07/2020,http://securities.stanford.edu/filings-documen...
1,2,U.S. District Court Civil Docket,07/08/2020,http://securities.stanford.edu/filings-documen...


Start Downloading!

In [172]:
# Set Output Folder location
output_folder = "/Volumes/yz-drive/class-action-filings-all"

In [173]:
# Function to download a pdf from a link
def url_response(url):
    path, url = url
    r = requests.get(url, stream = True)
    
    with open(output_folder + "/" + path,"wb") as pdf: 
        for chunk in r.iter_content(chunk_size=1024): 

            # writing one chunk at a time to pdf file 
            if chunk: 
                 pdf.write(chunk)

In [185]:
# Function to bulk download
#filingsDic = {}
filingsDicAll = {}
def downloadAll(data, dic, option="all"):
    
    #loop through data (scraped_fic)
    for x,y in data.iterrows():
        #save subtable
        table = pd.read_json(data["fic_summary_table"][x])
        print(x)
        
        #save unique identifier  
        identifier = data["id"][x]
        
        #loop through each filing in fic_summary_table
        for i, j in table.iterrows():
            
            #save title and date to be parts of the dictionary and file name
            if option == "all":
                no = str(table["No"][i])
            else:
                no = ""
            title = table["Document Title"][i][0:150].replace("/","")
            date = table["Filing Date"][i].replace("/","")
            filename = identifier + "_" + no + title + "_" + date + ".pdf"
            print(filename)
            
            #duplicates management
            ##create new key only when identifer is not a key already
            if identifier not in dic:
                dic[identifier] = [filename]
            ##else only append when file name is not the value of the identifier
            elif filename not in dic[identifier]:
                dic[identifier].append(filename)
            
            url_response([filename, table["link"][i]])

In [197]:
# Track the processing of downloading
print("Already downloaded: " + str(len(filingsDicAll.keys())) + " out of " + str(len(scraped_fic)))

Already downloaded: 5254 out of 5254


In [198]:
# Download all the files!
#downloadAll(scraped_fic,filingsDicAll, option = "all")

Check if All files are downloaded correctly

In [221]:
# Count How Many documents are there
l = []
for x in filingsDicAll:
    l.extend(filingsDicAll[x])
len(l)

16826

In [210]:
# Finding duplicates with same id, title and date
nodup = [] 
dup = []
for x,y in scraped_fic.iterrows():
    #save subtable
    table = pd.read_json(scraped_fic["fic_summary_table"][x])
    identifier = scraped_fic["id"][x]
    
    for i, j in table.iterrows():
        title = table["Document Title"][i].replace("/","")
        date = table["Filing Date"][i].replace("/","")
        filename = identifier + "_" + title + "_" + date + ".pdf"
        
        if filename not in nodup:
            nodup.append(filename)
        else:
            dup.append(filename)

In [222]:
len(dup), len(nodup)

(88, 16738)

In [219]:
pd.DataFrame(dup).to_csv("../../data/duplist.csv")

In [223]:
# Output saved dictionary for tacking purpuses later
with open('../../data/filingsDicAllIndent4', 'w') as json_file:
    json.dump(filingsDicAll, json_file, indent = 4)